In [ ]:
!pip install konlpy

In [ ]:
import xlrd
import openpyxl
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
import gensim
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt
import os

# Replace 'your_directory_path' with the path of your desired directory
directory_path = 'G:/academic/'

# Change the working directory to the specified directory path
os.chdir(directory_path)

# Verify that the working directory has been changed by printing the current directory
print(os.getcwd())

In [ ]:
# Replace 'your_excel_file.xlsx' with the name or path of your Excel file
file_path = 'data/data_name.xlsx'

# Read the Excel file and load it into a DataFrame
df = pd.read_excel(file_path, engine='openpyxl')

# Display the first few rows of the DataFrame
df.head()

In [ ]:
print("null exist:", df.isnull().values.any()) # Null check
df = df.dropna(how = 'any') # Null delete
print("null exist(renew):", df.isnull().values.any()) # Null re-check
print("number of title:", len(df)) # counting sentense

In [ ]:
# Remove non-Korean characters using regular expressions
df['title'] = df['title'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
print(df[:5])    # 상위 5개 출력

# Definition of stopwords
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','서','대다','로','에서','등','까지']


In [ ]:
# tokenizer OKT(Open Korean Text)
okt = Okt()
tokenized_data = []
for sentence in df['title']:
    temp_X = okt.morphs(sentence, stem=True) # tokenizing
    temp_X = [word for word in temp_X if not word in stopwords] # stopwords
    tokenized_data.append(temp_X)
print(tokenized_data)


In [ ]:
print('Max Length :',max(len(l) for l in tokenized_data))
print('Mean Length :',sum(map(len, tokenized_data))/len(tokenized_data))
plt.hist([len(s) for s in tokenized_data], bins=50)
plt.xlabel('Length of title')
plt.ylabel('Frequency')
plt.show()

In [ ]:
#Word2Vec model
from gensim.models import Word2Vec
model = Word2Vec(sentences = tokenized_data, vector_size = 100, window = 10, min_count = 5, workers = 100, epochs=300, sg = 1) #sg=1 skipgram #vector_size or size # iter or epoch

In [ ]:
# embedded matrix
print("Embedded matrix:", model.wv.vectors.shape)
print(model.wv.most_similar('위기', topn=10))   #top 10 results
model.wv.vectors

In [ ]:
from collections import Counter

# Assuming you have a list of tokenized sentences called 'tokenized_data'
word_counts = Counter([word for sentence in tokenized_data for word in sentence])

# Get the top 1000 most similar words to '위기'
wtw = model.wv.most_similar('위기', topn=1000)

# Filter out words with length 1 and get their frequencies
wtw_words = [word for word, score in wtw if len(word) > 1]
wtw_scores = [score for word, score in wtw if len(word) > 1]
wtw_lengths = [len(word) for word, score in wtw if len(word) > 1]
wtw_frequencies = [word_counts[word] for word in wtw_words]

# Create a DataFrame with the word, similarity score, length, and frequency columns
dfwtw = pd.DataFrame({'Word': wtw_words, 'Similarity Score': wtw_scores, 'Length': wtw_lengths, 'Frequency': wtw_frequencies})

# Save the DataFrame to an Excel file
dfwtw.to_excel("neg_words_1000.xlsx", index=False, engine="openpyxl", encoding='utf-8')
